<a href="https://colab.research.google.com/github/mcagriaksoy/pollen_multi_class_ai/blob/main/Pollen_classification_TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
#Mehmet Cagri Aksoy . 2024
#github.com/mcagriaksoy

In [70]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [71]:
!unzip /content/drive/MyDrive/Datasets/Pollen/Pollen_Dataset.zip

Archive:  /content/drive/MyDrive/Datasets/Pollen/Pollen_Dataset.zip
replace Pollen Dataset/resize.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [72]:
CLASS_NUMBER = 77
channels = 3
img_height = 128
img_width = 128
epochs = 100
batch_size = 16

input_shape = (img_height,img_width,channels)

In [73]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# We'll build the CNN as a sequence of layers.
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
# Libraries to handle the data
import json
from PIL import Image # PIL = Python Image Library
from skimage.transform import resize
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D

In [74]:
img_augmented = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    height_shift_range=0.1,
    width_shift_range=0.1,
    brightness_range=(0.8,1.2),
    zoom_range = [1, 1.2],
    validation_split=0.2
)

dir = '/content/Pollen Dataset/train'

train_dataset = img_augmented.flow_from_directory(batch_size=batch_size,
                                                 directory=dir,
                                                 shuffle=False,
                                                 target_size=(img_height,img_width),
                                                 subset="training",
                                                 class_mode='categorical')

validation_dataset = img_augmented.flow_from_directory(batch_size=batch_size,
                                                 directory=dir,
                                                 shuffle=False,
                                                 target_size=(img_height,img_width),
                                                 subset="validation",
                                                 class_mode='categorical')


Found 2708 images belonging to 77 classes.
Found 675 images belonging to 77 classes.


In [75]:
base_model = tf.keras.applications.DenseNet121(input_shape = input_shape, include_top = False, weights = "imagenet")
base_model.trainable = False
model = tf.keras.Sequential([base_model,
                             tf.keras.layers.GlobalAveragePooling2D(),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(CLASS_NUMBER, activation="softmax")
                            ])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 4, 4, 1024)        7037504   
                                                                 
 global_average_pooling2d_8  (None, 1024)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_8 (Dropout)         (None, 1024)              0         
                                                                 
 dense_8 (Dense)             (None, 77)                78925     
                                                                 
Total params: 7116429 (27.15 MB)
Trainable params: 78925 (308.30 KB)
Non-trainable params: 7037504 (26.85 MB)
_________________________________________________________________


In [76]:
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [77]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

model_path = 'best_model.h5'
callback = [EarlyStopping(monitor ='val_loss', patience = 25),
             ModelCheckpoint(filepath = model_path, save_best_only = True)]

In [ ]:
model.fit(
    train_dataset,
    steps_per_epoch = train_dataset.samples // batch_size,
    validation_data = validation_dataset,
    validation_steps = validation_dataset.samples // batch_size,
    epochs = 200,
    callbacks = callback
    )

Epoch 1/200
 17/169 [==>...........................] - ETA: 1:56 - loss: 5.9114 - accuracy: 0.0147